This script walks through a forecast for SMR using the NN-operational model developed in the NASA-NW repo. Using just observed data to forecast the next 7 days with only autoregressive temperature being updated for each forecast day.

# Import Modules

In [2]:
#high level modules
import os
import imp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

/var/folders/x8/v7bmckc139v2dqcm04jgfjlr0000gn/T/ipykernel_3636/3126687933.py:3: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  import imp


In [3]:
# custom modules
this_dir = "/Users/steeleb/Documents/GitHub/ATS-data-driven-forecasting/NN-operational/"

imp.load_source("universals", os.path.join(this_dir, "universal_functions.py"))
from universals import load_pickle_file, calculate_vals


# Import models

In [4]:
model_dir = "/Users/steeleb/Documents/GitHub/ats-data-driven-forecasting/data/NN_train_val_test/SMR_forecast/models/leaky_5_summary_t2023/"

model_1 = load_pickle_file("model_1.pkl", model_dir)
model_2 = load_pickle_file("model_2.pkl", model_dir)
model_3 = load_pickle_file("model_3.pkl", model_dir)
model_4 = load_pickle_file("model_4.pkl", model_dir)
model_5 = load_pickle_file("model_5.pkl", model_dir)
model_6 = load_pickle_file("model_6.pkl", model_dir)
model_7 = load_pickle_file("model_7.pkl", model_dir)
model_8 = load_pickle_file("model_8.pkl", model_dir)
model_9 = load_pickle_file("model_9.pkl", model_dir)


# Import data

In [5]:
data_dir = "/Users/steeleb/Documents/GitHub/ats-data-driven-forecasting/data/NN_train_val_test/SMR_forecast/met_summary/"

test = pd.read_csv(os.path.join(data_dir, "t2023_standardized_summary_v2024-11-29.csv"))

test["date"] = pd.to_datetime(test["date"])

# we need the test columns to be the same as the forecast columns at the end of this, so grab the names for now
forecast_cols = test.columns

# and let's drop the observed temp data from the forecast columns, too
forecast_cols_less = forecast_cols.drop(["mean_1m_temp_degC", "mean_0_5m_temp_degC", "date"])

# Create function to roll out forecast

This implementation just checks to see if, with observed data, we have any accuracy on a 7-day rollout.

In [6]:
def make_forecast(features, models, valid_date):
    predictions = pd.DataFrame(columns=['mean_1m_temp_degC', 'mean_0_5m_temp_degC', 'model', 'valid_date'])
    for i, model in enumerate(models, start=1):
        preds = {}
        preds = model.predict(features)
        temp_df = pd.DataFrame(preds, columns=['mean_1m_temp_degC', 'mean_0_5m_temp_degC'])
        for col in temp_df.columns:
            temp_df[col] = (temp_df[col])
        temp_df['model'] = i
        temp_df['valid_date'] = valid_date
        predictions = pd.concat([predictions, temp_df], ignore_index=True)
    return predictions

def make_forecast_by_model(features, models, valid_date):
    predictions = pd.DataFrame(columns=['mean_1m_temp_degC', 'mean_0_5m_temp_degC', 'model', 'valid_date'])
    for i, model in enumerate(models, start=1):
        preds = {}
        model_features = features.iloc[[i-1]]
        preds = model.predict(model_features)
        temp_df = pd.DataFrame(preds, columns=['mean_1m_temp_degC', 'mean_0_5m_temp_degC'])
        temp_df['model'] = i
        temp_df['valid_date'] = valid_date
        predictions = pd.concat([predictions, temp_df], ignore_index=True)
    return predictions

In [7]:

def make_seven_day_forecast_control(date):

    print(f"Beginning forecast for {date}")
    date = pd.to_datetime(date)

    # create a dataframe to store the forecasted data
    all_forecasts = pd.DataFrame(columns=['valid_date', 'model', 'mean_1m_temp_degC', 'mean_0_5m_temp_degC'])

    # we'll run a forecast for each day, since the following day's forecast will be based on the previous day's forecast
    for d in range(0, 7):
        
        # Setup for the iteration
        #print("Forecasting day: ", d+1)
        
        # set the forecast date
        valid_date = pd.to_datetime(date, utc=True) + pd.DateOffset(days=d)
        # subset the data for the day we're forecasting (this has been offset during the preprocessing steps)
        obs = test[test["date"] == valid_date].copy()
        
        # the first day will be a bit different from subsequent days
        if d == 0:

            # preprocess the data into labels and features
            features = obs.drop(columns = ["date", "mean_1m_temp_degC", "mean_0_5m_temp_degC"])

            forecasted_temp = (make_forecast(features, 
                                            [model_1, model_2, model_3, model_4, model_5, model_6, model_7, model_8, model_9], 
                                            valid_date))

        elif d >= 1:
            # remove the labels and observed temperature data from yesterday and today (we'll replaced these with forecasted data)
            obs_less = obs.drop(columns=["mean_1m_temp_degC", "mean_0_5m_temp_degC", 
                                            "mean_1m_temp_degC_m1", "mean_0_5m_temp_degC_m1"])
            # this should be one row of data
            
            # join the observed data with the forecasted data from the previous day by date
            yesterday = all_forecasts.copy()
            # get yesterday's forecasted data (valid date minus 1 day)
            yesterday = yesterday[yesterday["valid_date"] == valid_date - pd.DateOffset(days=1)]
            # relabel yesterday's forecasted data as today's m1 data
            yesterday = yesterday.rename(columns={"valid_date": "date",
                                                    "mean_1m_temp_degC": "mean_1m_temp_degC_m1", 
                                                    "mean_0_5m_temp_degC": "mean_0_5m_temp_degC_m1"})
            
            # now, set the date to today to join
            yesterday["date"] = pd.to_datetime(yesterday["date"], utc=True) + pd.DateOffset(days=1)
            yesterday = yesterday.set_index(["date"])
            
            # join the observed data with the forecasted data from the previous day by date
            to_forecast = obs_less.join(yesterday, on=["date"])
            
            # now we need to reorganize the columns to match the input columns, plus the model and peturbation colums
            to_forecast = to_forecast.reset_index()
            # now change model and date to the index
            to_forecast = to_forecast.set_index(["date", "model"])
            # and now reorganize the columns to match the input columns
            to_forecast = to_forecast[forecast_cols_less]

            # and now forecast on that data
            forecasted_temp = (make_forecast_by_model(to_forecast, 
                                                     [model_1, model_2, model_3, model_4, model_5, model_6, model_7, model_8, model_9], 
                                                     valid_date))

        # append the forecasted data to the all_forecasts dataframe
        all_forecasts = pd.concat([all_forecasts, forecasted_temp])
    
    return all_forecasts


# Make forecasts

In [8]:
datesequence = pd.date_range(start="2023-06-01", end="2023-09-01", freq="1D")
mean_std = pd.read_csv(os.path.join(data_dir, "mean_std_train_val_summary_t2023_v2024-11-29.csv"))
mean_std = mean_std.set_index("Unnamed: 0")

for d in datesequence:
    date = d.strftime("%Y-%m-%d")
    # make the forecast
    forecast = make_seven_day_forecast_control(d)
    # calculate the values back from the transformed values
    one_meter_transformed = calculate_vals(forecast["mean_1m_temp_degC"], mean_std.loc["mean_1m_temp_degC", "mean"], mean_std.loc["mean_1m_temp_degC", "std"])
    forecast["mean_1m_temp_degC"] = np.sqrt(one_meter_transformed)
    forecast["mean_0_5m_temp_degC"] = calculate_vals(forecast["mean_0_5m_temp_degC"], mean_std.loc["mean_0_5m_temp_degC", "mean"], mean_std.loc["mean_0_5m_temp_degC", "std"])
    # save the forecast
    forecast.to_csv(f"~/Documents/GitHub/ats-data-driven-forecasting/run-operational/output/control/{date}_seven_day_control.csv", index=False)
    

Beginning forecast for 2023-06-01 00:00:00


2024-12-01 20:46:54.594250: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
/var/folders/x8/v7bmckc139v2dqcm04jgfjlr0000gn/T/ipykernel_3636/3196941531.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  predictions = pd.concat([predictions, temp_df], ignore_index=True)
/var/folders/x8/v7bmckc139v2dqcm04jgfjlr0000gn/T/ipykernel_3636/2637449371.py:65: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_forecasts = pd.concat([all_forecasts, forecasted_tem

Beginning forecast for 2023-06-02 00:00:00


/var/folders/x8/v7bmckc139v2dqcm04jgfjlr0000gn/T/ipykernel_3636/3196941531.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  predictions = pd.concat([predictions, temp_df], ignore_index=True)
/var/folders/x8/v7bmckc139v2dqcm04jgfjlr0000gn/T/ipykernel_3636/2637449371.py:65: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_forecasts = pd.concat([all_forecasts, forecasted_temp])
/var/folders/x8/v7bmckc139v2dqcm04jgfjlr0000gn/T/ipykernel_3636/3196941531.py:23: FutureWarning: The behavior of Data

Beginning forecast for 2023-06-03 00:00:00


...

Beginning forecast for 2023-08-25 00:00:00


/var/folders/x8/v7bmckc139v2dqcm04jgfjlr0000gn/T/ipykernel_3636/2637449371.py:65: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_forecasts = pd.concat([all_forecasts, forecasted_temp])
/var/folders/x8/v7bmckc139v2dqcm04jgfjlr0000gn/T/ipykernel_3636/3196941531.py:23: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  predictions = pd.concat([predictions, temp_df], ignore_index=True)
/var/folders/x8/v7bmckc139v2dqcm04jgfjlr0000gn/T/ipykernel_3636/3196941531.py:23: FutureWarning: The behavior of Data

Beginning forecast for 2023-08-26 00:00:00


/var/folders/x8/v7bmckc139v2dqcm04jgfjlr0000gn/T/ipykernel_3636/3196941531.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  predictions = pd.concat([predictions, temp_df], ignore_index=True)
/var/folders/x8/v7bmckc139v2dqcm04jgfjlr0000gn/T/ipykernel_3636/2637449371.py:65: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_forecasts = pd.concat([all_forecasts, forecasted_temp])
/var/folders/x8/v7bmckc139v2dqcm04jgfjlr0000gn/T/ipykernel_3636/3196941531.py:23: FutureWarning: The behavior of Data

IndexError: positional indexers are out-of-bounds

In [10]:
datesequence = pd.date_range(start="2023-09-06", end="2023-10-15", freq="1D")
mean_std = pd.read_csv(os.path.join(data_dir, "mean_std_train_val_summary_t2023_v2024-11-29.csv"))
mean_std = mean_std.set_index("Unnamed: 0")

for d in datesequence:
    date = d.strftime("%Y-%m-%d")
    # make the forecast
    forecast = make_seven_day_forecast_control(d)
    # calculate the values back from the transformed values
    one_meter_transformed = calculate_vals(forecast["mean_1m_temp_degC"], mean_std.loc["mean_1m_temp_degC", "mean"], mean_std.loc["mean_1m_temp_degC", "std"])
    forecast["mean_1m_temp_degC"] = np.sqrt(one_meter_transformed)
    forecast["mean_0_5m_temp_degC"] = calculate_vals(forecast["mean_0_5m_temp_degC"], mean_std.loc["mean_0_5m_temp_degC", "mean"], mean_std.loc["mean_0_5m_temp_degC", "std"])
    # save the forecast
    forecast.to_csv(f"~/Documents/GitHub/ats-data-driven-forecasting/run-operational/output/control/{date}_seven_day_control.csv", index=False)
    

Beginning forecast for 2023-09-06 00:00:00


/var/folders/x8/v7bmckc139v2dqcm04jgfjlr0000gn/T/ipykernel_3636/3196941531.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  predictions = pd.concat([predictions, temp_df], ignore_index=True)
/var/folders/x8/v7bmckc139v2dqcm04jgfjlr0000gn/T/ipykernel_3636/2637449371.py:65: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_forecasts = pd.concat([all_forecasts, forecasted_temp])
/var/folders/x8/v7bmckc139v2dqcm04jgfjlr0000gn/T/ipykernel_3636/3196941531.py:23: FutureWarning: The behavior of Data

Beginning forecast for 2023-09-07 00:00:00


/var/folders/x8/v7bmckc139v2dqcm04jgfjlr0000gn/T/ipykernel_3636/2637449371.py:65: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_forecasts = pd.concat([all_forecasts, forecasted_temp])
/var/folders/x8/v7bmckc139v2dqcm04jgfjlr0000gn/T/ipykernel_3636/3196941531.py:23: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  predictions = pd.concat([predictions, temp_df], ignore_index=True)
/var/folders/x8/v7bmckc139v2dqcm04jgfjlr0000gn/T/ipykernel_3636/3196941531.py:23: FutureWarning: The behavior of Data

Beginning forecast for 2023-09-08 00:00:00


...

Beginning forecast for 2023-09-25 00:00:00


/var/folders/x8/v7bmckc139v2dqcm04jgfjlr0000gn/T/ipykernel_3636/3196941531.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  predictions = pd.concat([predictions, temp_df], ignore_index=True)
/var/folders/x8/v7bmckc139v2dqcm04jgfjlr0000gn/T/ipykernel_3636/2637449371.py:65: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_forecasts = pd.concat([all_forecasts, forecasted_temp])
/var/folders/x8/v7bmckc139v2dqcm04jgfjlr0000gn/T/ipykernel_3636/3196941531.py:23: FutureWarning: The behavior of Data

Beginning forecast for 2023-09-26 00:00:00


/var/folders/x8/v7bmckc139v2dqcm04jgfjlr0000gn/T/ipykernel_3636/2637449371.py:65: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_forecasts = pd.concat([all_forecasts, forecasted_temp])
/var/folders/x8/v7bmckc139v2dqcm04jgfjlr0000gn/T/ipykernel_3636/3196941531.py:23: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  predictions = pd.concat([predictions, temp_df], ignore_index=True)
/var/folders/x8/v7bmckc139v2dqcm04jgfjlr0000gn/T/ipykernel_3636/3196941531.py:23: FutureWarning: The behavior of Data

IndexError: positional indexers are out-of-bounds